In [1]:
import pandas as pd
import re

In [3]:
df = pd.read_csv("./../../data_llm_bs.csv")
df.head()

,Unnamed: 0,created_at,text,handle,displayname,like,repost,reply
0,0,2025-03-25T07:55:40.962Z,🚨Voici un outil Open Source très intéressant e...,duggytuxy.bsky.social,Laurent M.,1,0,0
1,1,2025-03-24T22:14:22.627Z,“If I had to speculate” proceeds to continue b...,ilarias.bsky.social,Ilaria 🇺🇦🇵🇱🇱🇹,1,0,1
2,2,2025-03-24T21:51:17.024Z,Modest proposal on a fix to this (𝘳𝘦𝘱𝘭𝘢𝘤𝘦 𝘵𝘩𝘦 ...,rogntudju.bsky.social,𓃰 Rogntuudju 𓃰,1,0,0
3,3,2025-03-24T20:59:01.804Z,Et je te parle même pas des groupes plus ou mo...,akbida.bsky.social,Abdellatif Kbida,1,0,1
4,4,2025-03-24T18:04:17.512Z,« les constatations humaines » : donc ils bluf...,caligans.bsky.social,phantôme,0,0,1


## Regex

In [4]:
re.findall?

Signature: re.findall(pattern, string, flags=0)
Docstring:
Return a list of all non-overlapping matches in the string.

If one or more capturing groups are present in the pattern, return
a list of groups; this will be a list of tuples if the pattern
has more than one group.

Empty matches are included in the result.
File:      /opt/anaconda3/lib/python3.11/re/__init__.py
Type:      function

In [14]:
mask = r"\b(\d{4})\b"

- on veut quatre caractères numériques à la suite : `\d{4}`
- on veut que la regex renvoie que cet élément : `(\d{4})`
- on veut que ces quatre caractères soit en bordure de part et d'autre : `\b(\d{4})\b`

In [15]:
re.findall(mask,"Ceci est une année 2342, et ceci est une autre 3323")

['2342', '3323']

In [20]:
df["text"].apply(lambda x : re.findall(mask, x))

0           []
1           []
2           []
3           []
4           []
         ...  
2394        []
2395        []
2396        []
2397    [2023]
2398        []
Name: text, Length: 2399, dtype: object

In [17]:
df["text"].str.extractall(mask)

,,0
,match,
6,0,2025
23,0,2025
40,0,2025
50,0,1300
56,0,2020
...,...,...
2329,0,2309
2339,0,2307
2384,0,3000


## Découper un texte en mots

In [22]:
texte = "il fait beau aujourd'hui"
mots = re.split(r"\W+", texte)
mots

['il', 'fait', 'beau', 'aujourd', 'hui']

## Contruire la matrice document/terme

In [25]:
#pip install scikit-learn

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

In [27]:
CountVectorizer

sklearn.feature_extraction.text.CountVectorizer

In [28]:
modele = CountVectorizer(min_df=5)

In [39]:
modele.fit(df["text"])

CountVectorizer(min_df=5)

In [35]:
text_tranform  = modele.transform(df["text"])

In [45]:
pd.DataFrame(text_tranform.toarray(), 
             columns= modele.get_feature_names_out())

,000,01,02,05,10,100,10x,11,12,13,...,étudiants,été,évaluation,évidemment,évident,éviter,évoluer,évolution,êtes,être
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2394,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2395,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2396,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Découverte de spacy

In [47]:
#pip install spacy

In [49]:
#!python -m spacy download fr_core_news_md

In [50]:
import spacy

In [51]:
modele = spacy.load("fr_core_news_md")

In [54]:
texte_transforme = modele("Ceci est un texte")

In [55]:
type(texte_transforme)

spacy.tokens.doc.Doc

In [60]:
mot = texte_transforme[1]

In [64]:
for token in texte_transforme:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

Ceci ceci PRON PRON nsubj Xxxx True True
est être AUX AUX cop xxx True True
un un DET DET det xx True True
texte texte NOUN NOUN ROOT xxxx True False


In [65]:
def filtrer_verbes(texte):
    texte_transforme = modele(texte)
    return [mot.text for mot in texte_transforme if mot.pos_ == "NOUN"]

In [66]:
filtrer_verbes("Ceci est un texte")

['texte']

In [67]:
df["text"].apply(filtrer_verbes)

0       [outil, Open, Source, ℹ️Garak, recherche, fuit...
1       [proceeds, baselessly, paragraphs, garbanzo, m...
2                                     [𝘳𝘦𝘱𝘭𝘢𝘤𝘦, 𝘓𝘓𝘔, 𝘓𝘓𝘔]
3       [groupes, blogs, LLM, vessies, lanternes, fois...
4                          [constatations, LLM, rapports]
                              ...                        
2394           [caca, boîtes, soi, -, LLM, nuance, chose]
2395    [might, But, m, blogs, repo, concert, tools, S...
2396    [think, that’, B2B, transaction, value, But, n...
2397    [wrote, -, level, guide, novel, https://allenp...
2398                                [Pi, AI, bestie, LLM]
Name: text, Length: 2399, dtype: object

In [72]:
texte = "Vous finissez le semestre bientôt à Centrale Méditerrannée"

In [73]:
txt_t = modele(texte)

In [74]:
txt_t.ents

(Centrale Méditerrannée,)

In [75]:
df["text"].apply(lambda x: modele(x).ents)

0       ((Garak), (Vulnerability, Scanner), (ℹ️Garak),...
1       ((baselessly, speculating, as, he, did, for, p...
2                                             ((Modest),)
3                                 ((empoissonner), (IAG))
4                                                      ()
                              ...                        
2394                        ((Elon), (Twitter), (Google))
2395    ((Oreilly, books, and, trying, to, do, stuff),...
2396                 ((I), (B2B, transaction, that, has))
2397                                               ((I),)
2398                       ((my, personal, AI), (#, LLM))
Name: text, Length: 2399, dtype: object

## Utiliser Gliner pour du NER

In [77]:
#pip install gliner

In [ ]:
model = GLiNER.from_pretrained("urchade/gliner_multi_pii-v1")

In [7]:
texte = "During my lunch, I ate rice and leeks salad"
model.predict_entities(texte, ["food"])

[{'start': 23,
  'end': 43,
  'text': 'rice and leeks salad',
  'label': 'food',
  'score': 0.9316188097000122}]

In [1]:
from gliner import GLiNER


text = """
Harilala Rasoanaivo, un homme d'affaires local d'Antananarivo, a enregistré une nouvelle société nommée "Rasoanaivo Enterprises" au Lot II M 92 Antohomadinika. Son numéro est le +261 32 22 345 67, et son adresse électronique est harilala.rasoanaivo@telma.mg. Il a fourni son numéro de sécu 501-02-1234 pour l'enregistrement.
"""

labels = ["work", "booking number", "personally identifiable information", "driver licence", "person", "book", "full address", "company", "actor", "character", "email", "passport number", "Social Security Number", "phone number"]
entities = model.predict_entities(text, labels)

for entity in entities:
    print(entity["text"], "=>", entity["label"])

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model.bin:  30%|##9       | 346M/1.16G [00:00<?, ?B/s]

/opt/anaconda3/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Harilala Rasoanaivo => person
Rasoanaivo Enterprises => company
Lot II M 92 Antohomadinika => full address
+261 32 22 345 67 => phone number
harilala.rasoanaivo@telma.mg => email
501-02-1234 => Social Security Number


In [ ]:
#pip install -U gliner